In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse
import json

In [2]:
news = pd.read_csv("uci-news-aggregator.csv")

# utils

In [3]:
def remove_www(x):
    dot_split = x.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return x

# cleaning list

In [4]:
news["HOSTNAME"] = news.HOSTNAME.apply(remove_www)

In [5]:
news = news.drop_duplicates(subset="HOSTNAME", keep="last")

In [6]:
news_domains = news.HOSTNAME.unique().tolist()

In [7]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9549

In [8]:
for i in news_domains:
    if ".edu" in i:
        print(i)

hawaii.edu
uconn.edu
buffalo.edu
iwu.edu
uiowa.edu


In [9]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9545

In [10]:
to_rem = [
    "youtube.com",
    "co.ke",
    "co.nz",
    "co.tt",
    "co.tz",
    "co.ug",
    "co.uk",
    "co.za",
    "com.au",
    "com.lb",
    "com.ng",
    "com.ph",
    "com.pk",
    "com.sg",
    "com.ua",
    "linkedin.com",
]

In [11]:
news_domains = [x for x in news_domains if x not in to_rem]
len(news_domains)

9529

In [12]:
to_add = ["techmeme.com", "news.sky.com"]
for i in to_add:
    news_domains.append(i)
len(news_domains)

9531

In [13]:
with open('news_domains.txt', 'w') as f:
    f.write(json.dumps(news_domains))